## Install necessary packages

In [ ]:
!pip install pillow pytesseract pyautogui pynput keyboard
!brew install tesseract

In [9]:
from PIL import Image, ImageGrab
import pytesseract
import pyautogui
import requests
from pynput.mouse import Listener


## Screen capture part
User can click on 2 positions on the screen and ImageGrab capture the rectangle defined by those coordinates

In [10]:
class ClickListener:
    def __init__(self):
        self.coordinates = None
        self.keep_listening = True

    def on_click(self, x, y, button, pressed):
        if pressed:
            self.coordinates = int(x), int(y)
            self.keep_listening = False
            return False  # Stop the listener

    def get_coordinates(self):
        with Listener(on_click=self.on_click) as listener:
            listener.join()  # This will block until on_click returns False
        return self.coordinates


In [25]:
def capture_screen():
    # Create an instance of ClickListener
    click_listener = ClickListener()
    
    # Get the mouse coordinates after the first click
    x1, y1 = click_listener.get_coordinates()
    x2, y2 = click_listener.get_coordinates()
    
    # Print the coordinates
    # print("First click coordinates:", x1, y1)
    print(f'Rectangle: x1: {x1} y1: {y1} x2: {x2} y2: {y2}')
    try:
        image = ImageGrab.grab(bbox=(x1, y1, x2, y2))
    except:
        return None
    return image

In [12]:

# image = capture_screen()
# display(image)

## Image to Text
Use Tesseract to recognize text on the image and convert it to a default prompt.
This prompt can be sent to the defined LLM to seek help. LLM can run local or can be a paid service like OpenAi or it can have an escalation from small local model -> organization hosted medium model -> very large LLM.

In [13]:
# text = pytesseract.image_to_string(image)
# # And with this text you can call the LLM
# PROMPT_DEFAULT = "Help me with this: \n"
# prompt = PROMPT_DEFAULT + text
# print(prompt)

## Integrate with LM Studio (https://lmstudio.ai/)
- Start LM studio with a model of your choice locally.
- Start the Local Inference Server

In [26]:
def send_request(user_input):
        url = f"http://localhost:1234/v1/chat/completions"
        headers = {"Content-Type": "application/json"}
        data = {
            "messages": [
                {"role": "user", "content": f"### Instruction: {user_input}\n###Response: "}
            ],
            "stop": ["### Instruction:"],
            "temperature": 0.7,
            "max_tokens": -1,
            "stream": False
        }
        response = requests.post(url, headers=headers, json=data)
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']


In [15]:
# response = send_request(prompt)
# print(response)

In [27]:
def help_me():
    image = capture_screen()
    if image != None:
        text = pytesseract.image_to_string(image)
        # PROMPT_DEFAULT = "Summarize this: \n"
        PROMPT_DEFAULT = "Help me with this: \n"
        prompt = PROMPT_DEFAULT + text
        return send_request(prompt)
    pass

In [28]:
# help_me()

## Run a listener and activate capture and inference onde key combo ([Shift][A]) has pressed

In [ ]:
from pynput import keyboard
COMBINATIONS = [
    {keyboard.Key.shift, keyboard.KeyCode(char='a')},
    {keyboard.Key.shift, keyboard.KeyCode(char='A')},
    {keyboard.Key.shift, keyboard.KeyCode(char='x')},
    {keyboard.Key.shift, keyboard.KeyCode(char='X')}
]
# The currently active modifiers
current = set()
def on_press(key):
    if any([key in COMBO for COMBO in COMBINATIONS]):
        current.add(key)
        print(current)
        # if any(all(k in current for k in COMBO) for COMBO in COMBINATIONS):
        if any(x in current for x in {keyboard.KeyCode(char='A')}):
            print('Make a screenshot: define the screen area by click 2 orcners of a triange')
            print(help_me())
        # if key == keyboard.Key.shift and any(x in current for x in {keyboard.KeyCode(char='X')}):
        if any(x in current for x in {keyboard.KeyCode(char='X')}):
            print('Shift + X Apressed. Stopping the listener.')
            listener.stop()

def on_release(key):
    try:
        if any([key in COMBO for COMBO in COMBINATIONS]):
            current.remove(key)
    except:
        pass
    # if key == keyboard.Key.shift and any(x in current for x in {keyboard.KeyCode(char='X')}):
    #     print('Shift + X pressed. Stopping the listener.')
    #     listener.stop()

with keyboard.Listener(on_press=on_press, on_release=on_release) as listener:
    listener.join()

This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.
This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


{<Key.shift: <56>>}
{'A', <Key.shift: <56>>}
Make a screenshot: define the screen area by click 2 orcners of a triange


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


Rectangle: x1: 203 y1: 451 x2: 992 y2: 630


## 

## 

## This is just a dummy sandbox to provide some sort of GUI

In [ ]:
import tkinter as tk

In [ ]:
def print_to_new_window(text):
    new_window = tk.Toplevel(root)
    new_window.title("Prompt")

    text_label = tk.Label(new_window, text=text)
    text_label.pack()

# Create the main window
root = tk.Tk()
root.title("AI interact")


In [ ]:
print_to_new_window(prompt)

In [ ]:
root.mainloop()